In [18]:
!pip install biopython

In [19]:
from Bio import SeqIO
import csv

# Section 1
For Our dataset we need chromosome numbers along with exons, starting point of exon, endpoint of exon and the gene type (Transcript, long non-coding RNA etc). We have an annotated gtf file already. We just need to extract the requisite data from the file.

In [20]:
exon_count = 0

with open('D:/kamran/gencode.v42.annotation.gtf/gencode.v42.annotation.gtf', 'r') as gtf_file:
    for line in gtf_file:
        if line.startswith('#'):
            continue
        fields = line.strip().split('\t')
        if fields[2] == 'exon':
            exon_count += 1

print('Number of exons:', exon_count)

Number of exons: 1643237


In [21]:
# Step 1: Parse GTF file and extract relevant information
gtf_file = open('D:/kamran/gencode.v42.annotation.gtf/gencode.v42.annotation.gtf', 'r')
# Create a dictionary to map chromosome names
chrom_mapping = {
    'chr1': '1',
    'chr2': '2',
    'chr3': '3',
    'chr4': '4',
    'chr5': '5',
    'chr6': '6',
    'chr7': '7',
    'chr8': '8',
    'chr9': '9',
    'chr10': '10',
    'chr11': '11',
    'chr12': '12',
    'chr13': '13',
    'chr14': '14',
    'chr15': '15',
    'chr16': '16',
    'chr17': '17',
    'chr18': '18',
    'chr19': '19',
    'chr20': '20',
    'chr21': '21',
    'chr22': '22',
    'chrX': 'X',
    'chrY': 'Y',
    'chrM': 'MT'
}
data = []
reader = csv.reader(gtf_file, delimiter='\t')
for i in range(5):
  next(reader)
# Loop through GTF file and replace chromosome names
for row in reader:
    # Get chromosome name
    chrom = row[0]
    # Check if chromosome name needs to be mapped
    if chrom in chrom_mapping:
        # Map chromosome name
        chrom = chrom_mapping[chrom]
        # Update row with mapped chromosome name
        row[0] = chrom

    if row[2] == 'exon':
      chrom = row[0]
      start = int(row[3])
      end = int(row[4])
      gene_type = row[8].split(';')[2].split()[1].strip('"')
      data.append({'chrom': chrom, 'start': start, 'end': end, 'gene_type': gene_type})


In [22]:
data[1]

{'chrom': '1', 'start': 12613, 'end': 12721, 'gene_type': 'lncRNA'}

In [23]:
# Step 2: Parse FASTA file and extract DNA sequence for relevant regions
# fasta_file = open('D:/kamran/Homo_sapiens.GRCh38.dna.primary_assembly.fa/Homo_sapiens.GRCh38.dna.primary_assembly.fa', 'r')
# seq_dict = {}
# current_chrom = ''
# current_seq = ''
# for line in fasta_file:
#   if line.startswith('>'):
#       if current_chrom != '':
#           seq_dict[current_chrom] = current_seq
#       current_chrom = line[1:].split()[0].strip()
#       current_seq = ''
#   else:
#       current_seq += line.strip()
# seq_dict[current_chrom] = current_seq

In [24]:
from Bio import SeqIO

fasta_file_path = 'D:/kamran/Homo_sapiens.GRCh38.dna.primary_assembly.fa/Homo_sapiens.GRCh38.dna.primary_assembly.fa'
# Using Biopython's SeqIO to efficiently read the fasta file and create the sequence dictionary
seq_dict = {}
for record in SeqIO.parse(fasta_file_path, 'fasta'):
    seq_dict[record.id] = str(record.seq)


In [25]:
print(seq_dict.keys())

dict_keys(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9', 'MT', 'X', 'Y', 'KI270728.1', 'KI270727.1', 'KI270442.1', 'KI270729.1', 'GL000225.1', 'KI270743.1', 'GL000008.2', 'GL000009.2', 'KI270747.1', 'KI270722.1', 'GL000194.1', 'KI270742.1', 'GL000205.2', 'GL000195.1', 'KI270736.1', 'KI270733.1', 'GL000224.1', 'GL000219.1', 'KI270719.1', 'GL000216.2', 'KI270712.1', 'KI270706.1', 'KI270725.1', 'KI270744.1', 'KI270734.1', 'GL000213.1', 'GL000220.1', 'KI270715.1', 'GL000218.1', 'KI270749.1', 'KI270741.1', 'GL000221.1', 'KI270716.1', 'KI270731.1', 'KI270751.1', 'KI270750.1', 'KI270519.1', 'GL000214.1', 'KI270708.1', 'KI270730.1', 'KI270438.1', 'KI270737.1', 'KI270721.1', 'KI270738.1', 'KI270748.1', 'KI270435.1', 'GL000208.1', 'KI270538.1', 'KI270756.1', 'KI270739.1', 'KI270757.1', 'KI270709.1', 'KI270746.1', 'KI270753.1', 'KI270589.1', 'KI270726.1', 'KI270735.1', 'KI270711.1', 'KI270745.1', 'KI270714.1', 'KI270732.

In [26]:
len(seq_dict)

194

In [27]:
data[1]

{'chrom': '1', 'start': 12613, 'end': 12721, 'gene_type': 'lncRNA'}

In [28]:
# Step 3: Combine information from GTF and FASTA files
for record in data:
  chrom = record['chrom']
  start = record['start']
  end = record['end']
  gene_type = record['gene_type']
  seq = seq_dict[chrom][start-1:end]
  record['sequence'] = seq

In [29]:
data[1]

{'chrom': '1',
 'start': 12613,
 'end': 12721,
 'gene_type': 'lncRNA',
 'sequence': 'GTGTGTGGTGATGCCAGGCATGCCCTTCCCCAGCATCAGGTCTCCAGAGCTGCAGAAGACGACGGCCGACTTGGATCACACTCTTGTGAGTGTCCCCAGTGTTGCAGAG'}

In [30]:
import random

#Randomly sample 200000 rows from data dictionary
#small_data = random.sample(data, 200000)

# Print the length of the subset dictionary
#print(len(small_data))


In [31]:
import pandas as pd

df = pd.DataFrame.from_dict(data)

In [32]:
df.head()

,chrom,start,end,gene_type,sequence
0,1,11869,12227,lncRNA,GTTAACTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCAT...
1,1,12613,12721,lncRNA,GTGTGTGGTGATGCCAGGCATGCCCTTCCCCAGCATCAGGTCTCCA...
2,1,13221,14409,lncRNA,GCAGGGCCATCAGGCACCAAAGGGATTCTGCCAGCATAGTGCTCCT...
3,1,12010,12057,transcribed_unprocessed_pseudogene,GTGTCTGACTTCCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGCTGAG
4,1,12179,12227,transcribed_unprocessed_pseudogene,TTGGAGGAAAGATGAGTGAGAGCATCAACTTCTCTCACAACCTAGGCCA


In [33]:
#number of classes
gene_types = set(d['gene_type'] for d in data)
num_classes = len(gene_types)
print("Number of classes: ", num_classes)


Number of classes:  40


In [34]:
from collections import Counter

# Get counts of each class
class_counts = Counter(df['gene_type'])

# Get class representation and sort in descending order
class_reps = [(class_name, round(count / len(df) * 100, 4)) for class_name, count in class_counts.items()]
class_reps_sorted = sorted(class_reps, key=lambda x: x[1], reverse=True)

# Print representation of each class
for class_name, class_rep in class_reps_sorted:
    print(f"{class_name}: {class_rep}%")


protein_coding: 84.1407%
lncRNA: 13.1681%
processed_pseudogene: 0.7164%
unprocessed_pseudogene: 0.476%
transcribed_unprocessed_pseudogene: 0.3601%
transcribed_unitary_pseudogene: 0.3315%
transcribed_processed_pseudogene: 0.1786%
misc_RNA: 0.1346%
snRNA: 0.1157%
miRNA: 0.1143%
TEC: 0.0659%
snoRNA: 0.0573%
rRNA_pseudogene: 0.0302%
unitary_pseudogene: 0.0217%
IG_V_pseudogene: 0.0175%
IG_V_gene: 0.0175%
TR_V_gene: 0.0133%
artifact: 0.0066%
IG_C_gene: 0.006%
TR_J_gene: 0.0048%
TR_V_pseudogene: 0.0035%
scaRNA: 0.003%
rRNA: 0.0029%
translated_unprocessed_pseudogene: 0.0029%
IG_D_gene: 0.0023%
pseudogene: 0.0021%
TR_C_gene: 0.0014%
Mt_tRNA: 0.0013%
IG_J_gene: 0.0011%
IG_C_pseudogene: 0.0009%
ribozyme: 0.0005%
sRNA: 0.0003%
translated_processed_pseudogene: 0.0002%
TR_D_gene: 0.0002%
TR_J_pseudogene: 0.0002%
IG_J_pseudogene: 0.0002%
scRNA: 0.0001%
vault_RNA: 0.0001%
IG_pseudogene: 0.0001%
Mt_rRNA: 0.0001%


In [35]:
  # Merge and rename the classes
df["gene_type"].replace({
    "processed_pseudogene": "pseudogene",
    "unprocessed_pseudogene": "pseudogene",
    "transcribed_unprocessed_pseudogene": "pseudogene",
    "transcribed_unitary_pseudogene": "pseudogene",
    "transcribed_processed_pseudogene": "pseudogene",
    "snRNA": "snRNA",
    "miRNA": "snRNA",
    "snoRNA": "snRNA"
}, inplace=True)

# Drop the underrepresented classes
df = df[df["gene_type"].isin(["protein_coding", "lncRNA", "pseudogene", "snRNA"])]

# Get the counts of each class
class_counts = df["gene_type"].value_counts()

# Print the counts
print(class_counts)


protein_coding    1382631
lncRNA             216383
pseudogene          33925
snRNA                4722
Name: gene_type, dtype: int64


In [36]:
import pandas as pd
from collections import Counter

# Get counts of each class
class_counts = Counter(df['gene_type'])

# Calculate class representation and put it in a DataFrame
class_df = pd.DataFrame({'gene_type': list(class_counts.keys()),
                         'count': list(class_counts.values())})
class_df['percentage'] = round(class_df['count'] / len(df) * 100, 4)

# Sort the DataFrame in descending order of percentage
class_df = class_df.sort_values(by='percentage', ascending=False)

# Display the DataFrame
#print(class_df)
#class_df.to_excel('class_distribution.xlsx', index=False)

#from google.colab import files
#files.download('class_distribution.xlsx')

In [37]:
#Representation of each class after merging
class_df

,gene_type,count,percentage
3,protein_coding,1382631,84.4272
0,lncRNA,216383,13.2129
1,pseudogene,33925,2.0716
2,snRNA,4722,0.2883


In [38]:
# initialize variables to hold largest and smallest sequences
largest_seq = ''
smallest_seq = ' ' * 1000  # set initial value to a long string

# iterate through data and find largest and smallest sequences
for record in data:
    seq = record['sequence']
    if len(seq) > len(largest_seq):
        largest_seq = seq
    if len(seq) < len(smallest_seq):
        smallest_seq = seq

# print results
print('Largest sequence:', len(largest_seq))
print('Smallest sequence:', len(smallest_seq))

Largest sequence: 347300
Smallest sequence: 1


In [39]:
# take smaller dataframe for testing purposes as the whole dataframe will crash the colab.
df = df.sample(n=200000, random_state=42)


In [40]:
#import random

#Randomly sample 200000 rows from data dictionary
#df = random.sample(data, 200000)

# Print the length of the subset dictionary
#print(len(df))


In [41]:
from sklearn.model_selection import train_test_split

X = df[['chrom', 'start', 'end', 'sequence']]
y = df['gene_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [42]:
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# One-hot encode the chromosome feature
enc = OneHotEncoder(sparse=False)
X_train_chrom = enc.fit_transform(X_train['chrom'].values.reshape(-1, 1))
X_test_chrom = enc.transform(X_test['chrom'].values.reshape(-1, 1))

# Tokenize the DNA sequence feature
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(X_train['sequence'])
X_train_seq = tokenizer.texts_to_sequences(X_train['sequence'])
X_test_seq = tokenizer.texts_to_sequences(X_test['sequence'])

# Pad the DNA sequence feature to a fixed length
max_seq_length = 100
X_train_seq = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_test_seq = pad_sequences(X_test_seq, maxlen=max_seq_length)


In [43]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_chrom.shape[1] + max_seq_length,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(df['gene_type'].unique()), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert gene_type labels to one-hot encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
le.fit(y_train)
y_train_enc = to_categorical(le.transform(y_train))
y_test_enc = to_categorical(le.transform(y_test))

history = model.fit(
    x=tf.concat([X_train_chrom, X_train_seq], axis=1),
    y=y_train_enc,
    validation_data=(tf.concat([X_test_chrom, X_test_seq], axis=1), y_test_enc),
    epochs=10,
    batch_size=32
)


Epoch 1/10
5000/5000 [==============================] - 3s 590us/step - loss: 0.5409 - accuracy: 0.8403 - val_loss: 0.5124 - val_accuracy: 0.8427
Epoch 2/10
5000/5000 [==============================] - 3s 561us/step - loss: 0.5138 - accuracy: 0.8422 - val_loss: 0.5098 - val_accuracy: 0.8427
Epoch 3/10
5000/5000 [==============================] - 3s 564us/step - loss: 0.5102 - accuracy: 0.8422 - val_loss: 0.5027 - val_accuracy: 0.8427
Epoch 4/10
5000/5000 [==============================] - 3s 589us/step - loss: 0.5090 - accuracy: 0.8422 - val_loss: 0.5057 - val_accuracy: 0.8427
Epoch 5/10
5000/5000 [==============================] - 3s 560us/step - loss: 0.5087 - accuracy: 0.8422 - val_loss: 0.5028 - val_accuracy: 0.8427
Epoch 6/10
5000/5000 [==============================] - 3s 576us/step - loss: 0.5079 - accuracy: 0.8422 - val_loss: 0.5019 - val_accuracy: 0.8427
Epoch 7/10
5000/5000 [==============================] - 3s 573us/step - loss: 0.5075 - accuracy: 0.8422 - val_loss: 0.5022 -

In [44]:
score = model.evaluate(tf.concat([X_test_chrom, X_test_seq], axis=1), y_test_enc, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Test loss: 0.5020818710327148
Test accuracy: 0.8427249789237976
